In [1]:
from CaltransTab import *
import requests
import os

In [2]:
county['ID'] = county['County'].str.replace(' ', '').str[:4]
county.head()

numbers = np.arange(0,10)
new_array = []

for id in county['ID']:
    for number in numbers:
        new_string = f"CA{id}{number}POP"
        new_array.append(new_string)

In [ ]:
for ID in new_array:
    params = {
        'bgcolor': '#e1e9f0',
        'chart_type': 'line',
        'drp': '0',
        'fo': 'open sans',
        'graph_bgcolor': '#ffffff',
        'height': '450',
        'mode': 'fred',
        'recession_bars': 'on',
        'txtcolor': '#444444',
        'ts': '12',
        'tts': '12',
        'width': '958',
        'nt': '0',
        'thu': '0',
        'trc': '0',
        'show_legend': 'yes',
        'show_axis_titles': 'yes',
        'show_tooltip': 'yes',
        'id': ID,
        'scale': 'left',
        'cosd': '1970-01-01',
        'coed': '2023-01-01',
        'line_color': '#4572a7',
        'link_values': 'false',
        'line_style': 'solid',
        'mark_type': 'none',
        'mw': '3',
        'lw': '2',
        'ost': '-99999',
        'oet': '99999',
        'mma': '0',
        'fml': 'a',
        'fq': 'Annual',
        'fam': 'avg',
        'fgst': 'lin',
        'fgsnd': '2020-02-01',
        'line_index': '1',
        'transformation': 'lin',
        'vintage_date': '2024-04-19',
        'revision_date': '2024-04-19',
        'nd': '1970-01-01',
    }

    response = requests.get('https://fred.stlouisfed.org/graph/fredgraph.xls', params=params)

    if len(response.content) < 20 * 1024:
        # Define the filename
        filename = f'cty_pop/{ID}.xls'

        with open(filename, 'wb') as file:
            file.write(response.content)
    
    print(f'{filename} ok')


In [22]:
# plt.plot(pd.read_excel('cty_pop/CABUTT5POP.xls').iloc[41:]['FRED Graph Observations'], 
#          pd.read_excel('cty_pop/CABUTT5POP.xls').iloc[41:]['Unnamed: 1'])

In [28]:
file_names = []
for i in new_array:
    file_name = f'cty_pop/{i}.xls'
    if os.path.isfile(file_name):
        #print(i)
        file_names.append(i)

file_names = sorted(list(set(file_names)))
county['FileName'] = file_names

In [29]:
percent_change = []
for i in county['FileName']:
    df = pd.read_excel(f'cty_pop/{i}.xls').iloc[41:]['Unnamed: 1'].reset_index(drop=True).to_frame().astype(float).rename(columns={'Unnamed: 1': 'Population'})
    df['% Change'] = df/(df.values[0]) - 1
    percent_change.append(df['% Change'].iloc[-1])

county['2001 to 2023'] = percent_change